In [32]:
# import dependencies
import os, torch, pandas as pd, numpy as np
from torch import nn
from torch.utils.data import DataLoader,Dataset
from torchvision.transforms import ToTensor
from torchvision.io import read_image

In [33]:
# Config
device = 'cuda' if torch.cuda.is_available() else 'cpu'
label_path = '/kaggle/input/location-detect/image_data2.csv'
img_root_path = '/kaggle/input/location-detect/Hotel1_image'
label_dic = {
    0:'Bedroom',
    1:'Diningroom',
    2:'Edwardian Suite',
    3:'Exterior',
    4:'Fitzgerald Suite',
    5:'Hardenbergh Terrace Suite',
    6:'Plaza Suite',
    
    8:'Royal Suite',
    9:'The Eloise Suite',
    10:'The Palm Court',
    11:'The Plaza Food Hall',
    12:'The Royal Plaza Suite'

}

In [34]:
class myImageDataset(Dataset):
    def __init__(self, img_dir, img_label_dir, transform=None):
        super().__init__()
        self.img_dir = img_dir
        self.img_labels = pd.read_csv(img_label_dir)  # 这是一个DataFrame，0是文件名，1是类别
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, index):
        label = self.img_labels.iloc[index, 1]
        img_path = os.path.join(self.img_dir + '/' + f'{label_dic[label]}' + '/' + self.img_labels.iloc[index, 0])
        image = read_image(img_path)
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [35]:
# CNN
class NormalCNN(nn.Module):
    def __init__(self):
        super(NormalCNN,self).__init__()

        # Flatten layer
        self.flatten = nn.Flatten()

        # Sequential layer
        self.sequential = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # FC
            nn.Linear(32 * 7 * 7, 120)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.sequential(x)
        return x

In [36]:
# SuperParameter
batch_size = 64
learning_rate = 1e-3

In [37]:
dataset = myImageDataset(img_root_path, label_path)
print(dataset[0:3])

TypeError: unhashable type: 'Series'